# Transformer Evaluation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
from pathlib import Path

wd = Path(os.path.dirname(os.path.abspath("__file__"))).parent.resolve()
sys.path.append(str(wd))

import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm
from datetime import datetime, timedelta
from src.models.auction_transformer import AuctionTransformer
from src.models.inference import predict_dataframe

pd.options.display.max_columns = None
pd.options.display.width = None

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [3]:
prediction_time = datetime.strptime("2025-03-20 00:00:00", "%Y-%m-%d %H:%M:%S")

mappings_dir = '../generated/mappings'

with open(os.path.join(mappings_dir, 'item_to_idx.json'), 'r') as f:
        item_to_idx = json.load(f)

with open(os.path.join(mappings_dir, 'context_to_idx.json'), 'r') as f:
    context_to_idx = json.load(f)
    
with open(os.path.join(mappings_dir, 'bonus_to_idx.json'), 'r') as f:
    bonus_to_idx = json.load(f)

with open(os.path.join(mappings_dir, 'modtype_to_idx.json'), 'r') as f:
    modtype_to_idx = json.load(f)

feature_stats = torch.load('../generated/feature_stats.pt')

time_left_mapping = {
    'VERY_LONG': 48,
    'LONG': 12,
    'MEDIUM': 2,
    'SHORT': 0.5
}

In [ ]:
def load_auctions_from_sample(data_dir, prediction_time):
    file_info = {}
    auction_appearances = {}

    for root, dirs, files in os.walk(data_dir):
        for filename in tqdm(files):
            filepath = os.path.join(root, filename)
            date = datetime.strptime(filename.split('.')[0], '%Y%m%dT%H')
            file_info[filepath] = date

    file_info = {k: v for k, v in sorted(file_info.items(), key=lambda item: item[1])}
    
    raw_auctions = []
    
    for filepath in list(file_info.keys()):
        with open(filepath, 'r') as f:
            try:
                json_data = json.load(f)
                
                if 'auctions' not in json_data:
                    print(f"File {filepath} does not contain 'auctions' key, skipping.")
                    continue
                
                auction_data = json_data['auctions']
                timestamp = file_info[filepath]
                
                for auction in auction_data:
                    auction_id = auction['id']

                    if auction_id not in auction_appearances:
                        auction_appearances[auction_id] = {'first': timestamp, 'last': timestamp}
                    else:
                        auction_appearances[auction_id]['last'] = timestamp
                
                if prediction_time is not None:
                    if timestamp == prediction_time:
                        raw_auctions.extend(auction_data)
                else:
                    raw_auctions.extend(auction_data)

            except json.JSONDecodeError as e:
                print(f"Error loading file {filepath}: {e}")
                continue
            except Exception as e:
                print(f"Unexpected error loading file {filepath}: {e}")
                continue

    auctions = []
    for auction in tqdm(raw_auctions):
        first_appearance = auction_appearances[auction['id']]['first']
        last_appearance = auction_appearances[auction['id']]['last']

        auction_id = auction['id']
        item_id = auction['item']['id']
        item_index = item_to_idx[str(item_id)]
        bid = auction.get('bid', 0) / 10000.0
        buyout = auction['buyout'] / 10000.0
        quantity = auction['quantity']
        time_left = time_left_mapping[auction['time_left']]
        context = context_to_idx[str(auction['item'].get('context', 0))]
        bonus_lists = [bonus_to_idx[str(bonus)] for bonus in auction['item'].get('bonus_lists', [])]
        modifiers = auction['item'].get('modifiers', [])

        modifier_types = []
        modifier_values = []

        for modifier in modifiers:
            modifier_types.append(modtype_to_idx[str(modifier['type'])])
            modifier_values.append(modifier['value'])

        if 'pet_species_id' in auction['item']:
            continue

        first_appearance = first_appearance.strftime('%Y-%m-%d %H:%M:%S')
        last_appearance = last_appearance.strftime('%Y-%m-%d %H:%M:%S')

        auctions.append([
            auction_id,
            item_index,
            bid,
            buyout,
            quantity,
            time_left,
            context,
            bonus_lists,
            modifier_types,
            modifier_values,
            first_appearance,
            last_appearance
        ])
        
    df_auctions = pd.DataFrame(auctions, columns=['id', 'item_index', 'bid', 'buyout', 'quantity', 'time_left', 'context', 'bonus_lists', 'modifier_types', 'modifier_values', 'first_appearance', 'last_appearance'])
    df_auctions['first_appearance'] = pd.to_datetime(df_auctions['first_appearance'])
    df_auctions['last_appearance'] = pd.to_datetime(df_auctions['last_appearance'])

    df_auctions = df_auctions[(df_auctions['first_appearance'] <= prediction_time) & (df_auctions['last_appearance'] >= prediction_time)]

    df_auctions['current_hours'] = (prediction_time - df_auctions['first_appearance']).dt.total_seconds() / 3600
    df_auctions['hours_on_sale'] = (df_auctions['last_appearance'] - prediction_time).dt.total_seconds() / 3600

    return df_auctions

data_dir = '../data/sample/'

df_auctions = load_auctions_from_sample(data_dir, prediction_time)

print("Auctions shape:", df_auctions.shape)
df_auctions.head()

In [ ]:
df_auctions[['current_hours', 'hours_on_sale']].describe()

In [ ]:
model = AuctionTransformer.load_from_checkpoint(
    '../models/auction_transformer_7.2M_128b_wpos_ch_filtered/last-v1.ckpt',
    map_location=device
)

print(f'Number of model parameters: {sum(p.numel() for p in model.parameters())}')
model.eval()
print('Pre-trained Transformer model loaded successfully.')

In [ ]:
from src.models.inference import predict_dataframe

model = model.to('cuda')
df_auctions = predict_dataframe(model, df_auctions, prediction_time, feature_stats)

print("Mean hours on sale:", df_auctions['hours_on_sale'].mean())
print("Mean prediction:", df_auctions['prediction'].mean())
print("Mean sale probability:", df_auctions['sale_probability'].mean())

df_auctions.head()

In [ ]:
df_auctions_12h = df_auctions[df_auctions['current_hours'] <= 12]
len(df_auctions_12h)

mae = mean_absolute_error(df_auctions_12h['hours_on_sale'], df_auctions_12h['prediction'])
print(f"Mean absolute error: {mae}")

In [ ]:
df_auctions.sample(5)

In [38]:
# save the dataframe to excel
df_auctions.to_excel('../generated/predictions.xlsx', index=False)

## Error analysis

In [ ]:
columns = [
    'item_name',
    'bid',
    'buyout',
    'quantity',
    'time_left',
    'first_appearance',
    'last_appearance',
    'hours_since_first_appearance',
    'hours_on_sale',
    'prediction',
    'sale_probability'
]

df_error = df_auctions[columns].copy()
df_error['error'] = np.abs(df_error['hours_on_sale'] - df_error['prediction'])
df_error['time_left'] = df_error['time_left'].map(time_left_mapping)

df_error.head()

In [ ]:
plt.boxplot(df_error[['hours_on_sale', 'prediction']])
plt.grid()
plt.show()

In [ ]:
# histogram of hours on sale and prediction
plt.hist(df_error['hours_on_sale'], bins=50, alpha=0.5, label='Hours on sale')
plt.hist(df_error['prediction'], bins=50, alpha=0.5, label='Prediction')
plt.legend(loc='upper right')
plt.grid()
plt.show()

In [ ]:
plt.hist(df_error[(df_error['time_left'] == 48.0) & (df_error['hours_since_first_appearance'] < 12)]['hours_on_sale'], bins=50, alpha=0.5, label='Hours on sale')
plt.hist(df_error[(df_error['time_left'] == 48.0) & (df_error['hours_since_first_appearance'] < 12)]['prediction'], bins=50, alpha=0.5, label='Prediction')
plt.legend(loc='upper right')
plt.grid()
plt.show()

In [ ]:
plt.hist(df_error['hours_since_first_appearance'], bins=15)
plt.grid()
plt.show()

We are interested in evaluating the model when the items are recently published, because this will be the main use case for the model

In [ ]:
query = (df_error['hours_since_first_appearance'] <= 12) & (df_error['hours_on_sale'] <= 5) & (df_error['time_left'] == 48.0)
query_df = df_error[query]
print(f"Mean sale probability: {query_df['sale_probability'].mean()}")
print(f"Mean error: {query_df['error'].mean()}")
print(f"Mean hours on sale: {query_df['hours_on_sale'].mean()}")
query_df.sample(10)

In [ ]:
query_df['hours_on_sale'].hist(bins=10)
plt.grid()
plt.show()

In [ ]:
corr_matrix = df_error[['bid', 'buyout', 'quantity', 'time_left', 'hours_since_first_appearance', 'sale_probability',
                        'hours_on_sale', 'prediction', 'error']].corr()

plt.figure(figsize=(10, 8))

sns.heatmap(corr_matrix, 
            annot=True, 
            cmap='coolwarm',
            vmin=-1, vmax=1, 
            center=0,
            fmt='.2f',
            square=True) 

plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)

plt.tight_layout()

plt.show()

In [18]:
df_error.to_csv('../generated/predictions.csv', index=False)

# Eval validation

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import sys
from tqdm import tqdm
from datetime import datetime
pd.options.mode.chained_assignment = None

from pathlib import Path

repo_root = Path.cwd().parent.resolve()
sys.path.append(str(repo_root))

from sklearn.model_selection import train_test_split
from src.data.auction_dataset import AuctionDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
pairs = pd.read_csv('../generated/auction_indices.csv')
pairs = pairs[pairs['g_hours_on_sale_max'] < 50]
pairs = pairs[pairs['g_current_hours_max'] < 50]

train_pairs, val_pairs = train_test_split(pairs, test_size=0.15, random_state=42, shuffle=False)

print(f"Before filtering: {len(train_pairs)}")

train_pairs = train_pairs[train_pairs['g_hours_on_sale_len'] <= 32]
val_pairs = val_pairs[val_pairs['g_hours_on_sale_len'] <= 32]

print(f"After filtering: {len(train_pairs)}\n")

train_pairs = train_pairs[:int(len(train_pairs)*0.9)]
val_pairs = val_pairs[val_pairs['record'] < '2025-03-23']

print(f"Train pairs: {len(train_pairs)}")
print(f"Val pairs: {len(val_pairs)}")

In [ ]:
val_pairs.head()

In [ ]:
val_pairs = val_pairs[val_pairs['record'] == '2025-03-20 00:00:00']
val_pairs.head()

In [44]:
import json
import os

mappings_dir = '../generated/mappings'

with open(os.path.join(mappings_dir, 'item_to_idx.json'), 'r') as f:
    item_to_idx = json.load(f)

with open(os.path.join(mappings_dir, 'context_to_idx.json'), 'r') as f:
    context_to_idx = json.load(f)
    
with open(os.path.join(mappings_dir, 'bonus_to_idx.json'), 'r') as f:
    bonus_to_idx = json.load(f)

with open(os.path.join(mappings_dir, 'modtype_to_idx.json'), 'r') as f:
    modtype_to_idx = json.load(f)

feature_stats = torch.load('../generated/feature_stats.pt')

In [ ]:
val_pairs.tail()

In [ ]:
model = AuctionTransformer.load_from_checkpoint(
    '../models/auction_transformer_7.2M_128b_wpos_ch_filtered/epoch_epoch=02.ckpt',
    map_location=device
)

print(f'Number of model parameters: {sum(p.numel() for p in model.parameters())}')
model.eval()
print('Pre-trained Transformer model loaded successfully.')

In [69]:
from src.data.auction_dataset import AuctionDataset
from src.data.utils import collate_auctions

batch_size = 32

val_dataset = AuctionDataset(val_pairs, feature_stats=feature_stats, path='../generated/sequences.h5')
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_auctions) 

a = next(iter(val_dataloader))

Dataset size: 15431


In [71]:
# evaluate the model on the validation set
total_mse = 0
total_mae = 0
total_samples = 0

with torch.no_grad():
    for batch in tqdm(val_dataloader):
        (auctions, item_index, contexts, bonus_lists, modifier_types, modifier_values, current_hours), y = batch

        auctions = auctions.to(device)
        item_index = item_index.to(device)
        contexts = contexts.to(device)
        bonus_lists = bonus_lists.to(device)
        modifier_types = modifier_types.to(device)
        modifier_values = modifier_values.to(device)
        current_hours = current_hours.to(device)
        y = y.to(device)
        
        #print(f'auctions {auctions[0]}')
        #print(f'item_index {item_index[0]}')
        #print(f'contexts {contexts[0]}')
        #print(f'bonus_lists {bonus_lists[0]}')
        #print(f'modifier_types {modifier_types[0]}')
        #print(f'modifier_values {modifier_values[0]}')

        model.eval()
        y_hat = model((auctions, item_index, contexts, bonus_lists, modifier_types, modifier_values))
        
        mask = (item_index != 0).float().unsqueeze(-1)
        current_hours_mask = (current_hours <= 12.0).float().unsqueeze(-1)
        mask = mask * current_hours_mask
        #print(item_index[0])
        #print(current_hours[0])
        
        mse = model.criterion(y_hat * mask, y.unsqueeze(2) * mask) / mask.sum()
        mae = torch.nn.functional.l1_loss(
            y_hat * mask * 48.0,
            y.unsqueeze(2) * mask * 48.0,
            reduction='sum'
        ) / mask.sum()

        #print((y_hat* 48.0 * mask).squeeze(2)[0])
        #print((y.unsqueeze(2) * mask * 48.0).squeeze(2)[0])
        
        total_mse += mse.item() * mask.sum()
        total_mae += mae.item() * mask.sum()
        total_samples += mask.sum()

        #break

avg_mse = total_mse / total_samples
avg_mae = total_mae / total_samples
print(f'Validation MSE: {avg_mse}')
print(f'Validation MAE: {avg_mae}')

100%|██████████| 483/483 [00:20<00:00, 23.69it/s]

Validation MSE: 0.06371152400970459
Validation MAE: 8.173335075378418


In [ ]:
df_auctions[df_auctions['item_index'] == 13815]

In [ ]:
from src.models.inference import predict_dataframe

predict_dataframe(model, df_auctions[df_auctions['item_index'] == 13815], prediction_time, feature_stats)

In [68]:
# open the file ../data/generated_test/sequences.h5 and print the auctions for item_index 2025-03-20/00/13815/auctions
with h5py.File('../generated/sequences.h5', 'r') as f:
    print(f['2025-03-20/00/7139/auctions'][:]) # there should be 8 auctions
    print(f['2025-03-20/00/7139/bonus_lists'][:])
    print(f['2025-03-20/00/7139/contexts'][:])
    print(f['2025-03-20/00/7139/modifier_types'][:])
    print(f['2025-03-20/00/7139/modifier_values'][:])


[[0.0000000e+00 1.0614372e+05 1.0000000e+00 4.8000000e+01 2.5000000e+01
  2.2000000e+01]
 [1.9588410e+04 3.6245220e+04 1.0000000e+00 4.8000000e+01 2.2000000e+01
  2.5000000e+01]
 [0.0000000e+00 3.6245220e+04 1.0000000e+00 4.8000000e+01 6.0000000e+00
  1.7000000e+01]]
[]
[0 0 0]
[[4.]
 [4.]
 [4.]]
[[1735.]
 [1735.]
 [1735.]]
